In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
import pack_ga
from IPython.display import HTML, display, clear_output
pack_cuda.USE_FLOAT32 = True
pack_cuda._ensure_initialized()


local
local
stop final relax at some point


In [ ]:
importlib.reload(pack_ga)
fastMode = False
runner = pack_ga.GA()
runner.N_trees_to_do = np.array([40,50])
if fastMode:
    runner.initializer.jiggler.duration_compact /= 10
    runner.initializer.jiggler.plot_interval = None
    runner.initializer.jiggler.n_rounds = 1
    runner.relaxers = runner.relaxers[:1]
    runner.relaxers[0].n_iterations= 10
    runner.population_size = 10
runner.run()

stop final relax at some point
Generation 0, Trees 40, Best cost: 0.4682
Generation 0, Trees 50, Best cost: 0.7493
Generation 1, Trees 40, Best cost: 0.4676
Generation 1, Trees 50, Best cost: 0.7458
Generation 2, Trees 40, Best cost: 0.4637
Generation 2, Trees 50, Best cost: 0.7159
Generation 3, Trees 40, Best cost: 0.4615
Generation 3, Trees 50, Best cost: 0.7100
Generation 4, Trees 40, Best cost: 0.4587
Generation 4, Trees 50, Best cost: 0.7062
Generation 5, Trees 40, Best cost: 0.4570
Generation 5, Trees 50, Best cost: 0.7029
Generation 6, Trees 40, Best cost: 0.4545
Generation 6, Trees 50, Best cost: 0.6999
Generation 7, Trees 40, Best cost: 0.4525
Generation 7, Trees 50, Best cost: 0.6963
Generation 8, Trees 40, Best cost: 0.4507
Generation 8, Trees 50, Best cost: 0.6933
Generation 9, Trees 40, Best cost: 0.4487
Generation 9, Trees 50, Best cost: 0.6899
Generation 10, Trees 40, Best cost: 0.4459
Generation 10, Trees 50, Best cost: 0.6869
Generation 11, Trees 40, Best cost: 0.4423
